<a href="https://colab.research.google.com/github/vaib-saxena/Slow-Fast-pytorch-implementation/blob/master/slow_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create a conda environment

In [0]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

## Install dependencies

In [0]:
!conda install pytorch==1.0.0 torchvision==0.2.1 cuda100 -c pytorch
!pip install opencv-python==3.4.5.20
!pip install -q youtube-dl
!pip install ffmpeg

## Clone the github repo

In [0]:
!git clone https://github.com/vaib-saxena/Slow-Fast-pytorch-implementation.git

In [0]:
%cd /content/Slow-Fast-pytorch-implementation

## Download pretrained weights

In [0]:
import os
from os.path import exists, join, basename
project_name = "/content/Slow-Fast-pytorch-implementation"

yolo_pretrained_weight_dir = project_name
if not exists(join(yolo_pretrained_weight_dir, 'yolov3.weights')):
  !cd {yolo_pretrained_weight_dir} && wget -q https://pjreddie.com/media/files/yolov3.weights
    
deepsort_pretrained_weight_dir = join(project_name, 'deep/checkpoint')
if not exists(join(deepsort_pretrained_weight_dir, 'ckpt.t7')):
  file_id = '1_qwTWdzT9dWNudpusgKavj_4elGgbkUN'
  !cd {deepsort_pretrained_weight_dir} && curl -Lb ./cookie "https://drive.google.com/uc?export=download&id={file_id}" -o ckpt.t7

slowfast_pretrained_weight_dir = project_name
if not exists(join(slowfast_pretrained_weight_dir, 'slowfast_weight.pth')):
  !cd {slowfast_pretrained_weight_dir} && curl https://uc131c6bc33bfb9f2b461fa93de9.dl.dropboxusercontent.com/cd/0/get/A2VLt5eNxIulWoLD2eHei2Cd48jYj4UK3JbT_YDyv-zJXeKl4Eylo-vb5KpPmXPyBqNUwhoesK_qNzshyj9xzEhIOVmKLP7_Va9hEh8Fh_oX5Zoq5F4JjvlAg8n9M4pgrEM/file?dl=1 -o slowfast_weight.pth

## Build the extension

In [0]:
!python /content/Slow-Fast-pytorch-implementation/support/setup.py build develop

In [0]:
%cd /content/Slow-Fast-pytorch-implementation

## Downloading a sample youtube video

In [0]:
from IPython.display import YouTubeVideo
#put your youtube id below
YOUTUBE_ID = 'Ftexl9o4QX8'
YouTubeVideo(YOUTUBE_ID)
!rm -rf youtube.mp4
# download the youtube with the given ID
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 10 seconds or more depending on your choice
!ffmpeg -ss 00:00:01.00 -i youtube.mp4 -t 00:00:15.00 -c copy new.mp4

## Run the code on the video(go get a coffee, will take time :P)

In [0]:
!python video_demo.py

## Creating the output as video in outputs folder(download it to view results)

In [0]:
import cv2
import numpy as np
import glob

path = '/content/Slow-Fast-pytorch-implementation/outputs/frames/'

img_array = []
for count in range(len(os.listdir(path))):
  filename = '/content/Slow-Fast-pytorch-implementation/outputs/frames/' + str(count) + '.jpg'
  img = cv2.imread(filename)
  height, width, layers = img.shape
  size = (width,height)
  img_array.append(img)

#set your own frame rate
frame = 15
out = cv2.VideoWriter('/content/Slow-Fast-pytorch-implementation/outputs/output.mp4',cv2.VideoWriter_fourcc(*'MJPG'), frame, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Display video

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('/content/Slow-Fast-pytorch-implementation/outputs/output.mp4', width=960, height=480)